<a href="https://colab.research.google.com/github/EricaAndreose/enoam_doc/blob/main/Recast_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re, unicodedata, unidecode
from bs4 import BeautifulSoup
from qwikidata.entity import WikidataItem
from qwikidata.linked_data_interface import get_entity_dict_from_api

Qui stiamo importando le seguenti librerie:


*   **re**: Per lavorare con le espressioni regolari.
*   **unicodedata**: Per lavorare con i dati Unicode.
*   **unidecode**: Per traslitterare i caratteri Unicode in caratteri ASCII.
*   **BeautifulSoup**: Per analizzare e fare parsing di documenti HTML.
*   **WikidataItem**: Per interagire con gli elementi di Wikidata.
*   **get_entity_dict_from_api**: Per ottenere i dati di un'entità da Wikidata tramite l'API.

In [ ]:
# CONVERT FAKE HEAD AND BODY INTO ACTUAL TAGS
def convert_head_body(old_soup, soup):  # it takes in input two BeautifulSoup objects
    headfile = old_soup.find('div', id='headFile') # searching for "div" elements with "headFile" as id
    bodyfile = old_soup.find('div', id='bodyFile') # searching for "div" elements with "bodyFile" as id
    if headfile:
        soup.head.insert(1, headfile) # Insert headfile into soup's head tag
        headfile.unwrap() # Remove headfile from its original position
    if bodyfile:
        soup.body.insert(1, bodyfile) # Insert bodyfile into soup's body tag
        bodyfile.unwrap() # Remove bodyfile from its original position

La funzione `convert_head_body` prende due oggetti BeautifulSoup come input: `old_soup` e `soup`.

Cerca all'interno di `old_soup` due elementi div con gli id "headFile" e "bodyFile". Se questi elementi vengono trovati, vengono inseriti rispettivamente all'interno dei tag `head` e `body` di `soup`, e poi vengono rimossi dalla loro posizione originale in `old_soup` usando il metodo `unwrap()`.

In pratica, questa funzione sposta il contenuto dei `div` "headFile" e "bodyFile" da `old_soup` ai tag `head` e `body` di `soup`, aggiornando la struttura del documento HTML.

❓ Dove prende tutte queste cose di input? Vai a vedere su main.py quando lanci "r" per recast.

In [ ]:
# INSERT META TAGS
def insert_seo_meta(soup, metadata): # takes in input the refined BS object and the metadata from MongoDB database
    split_ident = metadata['ident'].split('_') # split the 'ident' field of the metadata stored for every object in MongoDB (ex. 2_2_0_001)

    section = '' # create empty string variables
    volume = ''
    curators = ''
    if split_ident[0] == '1':
        section = 'Scritti e Discorsi'
        if split_ident[1] == '1':
            volume = 'Gli anni giovanili (1932-1946)'
            curators = 'Gaetano Crociata e Paolo Trionfini'
        elif split_ident[1] == '2':
            volume = 'Il periodo dossettiano e di Iniziativa democratica (1946-1958)'
            if split_ident[2] == '1':
                volume = 'Il periodo dossettiano (1946-1951)'
            elif split_ident[2] == '2':
                volume = 'Iniziativa democratica (1952-1958)'
            curators = 'Ugo De Siervo e Enrico Galavotti'
        elif split_ident[1] == '3':
            volume = 'Il centro-sinistra (1952-1958)'
            if split_ident[2] == '1':
                volume = 'Segretario della DC (1959-1963)'
            elif split_ident[2] == '2':
                volume = 'La prima legislatura di centro-sinistra (1964-1968)'
            curators = 'Leopoldo Nuti e Paolo Pombeni'
        elif split_ident[1] == '4':
            volume = 'L’ultima fase (1969-1978)'
            if split_ident[2] == '1':
                volume = 'Al ministero degli Esteri e all’opposizione nel partito (giugno 1968 – maggio 1973)'
            elif split_ident[2] == '2':
                volume = 'Il ritorno al centro-sinistra e la “solidarietà nazionale” (giugno 1973 – maggio 1978)'
            curators = 'Guido Formigoni e Agostino Giovagnoli'
    elif split_ident[0] == '2':
        section = 'Opere Giuridiche'
        if split_ident[1] == '1':
            volume = 'Le prime monografie (1939-1942)'
            curators = 'Luciano Eusebi'
        elif split_ident[1] == '2':
            volume = 'Le dispense di filosofia del diritto (1941-1947)'
            curators = 'Nicola Antonetti e Renato Moro'
        elif split_ident[1] == '3':
            volume = 'Le monografie del dopoguerra (1947-1951)'
            curators = 'Marco Pelissero'
        elif split_ident[1] == '4':
            volume = 'Le lezioni di istituzioni di diritto e procedura penale'
            curators = 'Marco Pelissero'

La funzione `insert_seo_meta` prende in input un oggetto BeautifulSoup (`soup`) e i metadati (`metadata`) di un documento. Lo scopo di questa funzione è quello di inserire meta tag nel documento HTML rappresentato da soup basandosi sui metadati forniti da MongoDB.

Inizialmente, l'identificativo del documento (`ident`) viene estratto dai metadati di MongoDB e suddiviso in parti (viene splittato). Queste parti vengono poi utilizzate per determinare la sezione, il volume e i curatori del documento.

La funzione crea nuovi tag meta per la sezione, il volume e i curatori, impostando il loro contenuto in base ai valori determinati in precedenza. Questi tag meta vengono quindi aggiunti all'head del documento HTML.

Qui è importante controllare la correttezza dei dati inseriti nel codice (sono già state fatte alcune modifiche in corso d'opera per la correzione di alcuni nomi di ricercatori). Nel caso di introduzione di nuovi sezioni/volumi nell'edizione digitale, vanno aggiunti qui.

Ciò che ci crea qui verrà visualizzato concretamente nel sito nella pagina del singolo documento nella sezione "metadati" e "citazione bibliografica".


In [ ]:
    description = soup.new_tag('meta') #create variable with new meta tag
    description['name'] = 'description' #create attribute name
    description['content'] = f"Edizione Nazionale delle Opere di Aldo Moro, {section}, {volume}, 2024" #create content of the meta tag taking the volum section infos from the variables from before

    rights = soup.new_tag('meta')
    rights['name'] = 'dcterms.rights'
    rights['content'] = 'https://creativecommons.org/licenses/by-nc/4.0'

    doi = soup.new_tag('meta')
    doi['name'] = 'dcterms.identifier'
    doi['content'] = f"10.48678/unibo/aldomoro{'.'.join(split_ident)}"

    url = soup.new_tag('meta')
    url['name'] = 'dcterms.relation'
    url['content'] = f"https://doi.org/{doi['content']}"

    citation = soup.new_tag('meta')
    citation['name'] = 'dcterms.bibliographicCitation'
    citation['content'] = f"Moro, Aldo, {metadata['title']}, in Aldo Moro, Edizione Nazionale delle Opere di Aldo Moro, Sezione {split_ident[0]}, {section}, Vol. {split_ident[1]}, {volume}, a cura di {curators}, edizione e nota storico-critica di {metadata['curator']}, Bologna, Università di Bologna, 2024. DOI: https://doi.org/10.48678/unibo/aldomoro{'.'.join(split_ident)}."

    viewport = soup.new_tag('meta')
    viewport['name'] = 'viewport'
    viewport['content'] = 'width=device-width, initial -scale=1.0'
    charset = soup.new_tag('meta')
    charset['charset'] = 'utf-8'

    soup.head.insert(0, description) #insert the new tags in the head section of the html
    soup.head.insert(0, citation)
    soup.head.insert(0, doi)
    soup.head.insert(0, url)
    soup.head.insert(0, rights)
    soup.head.insert(0, viewport)
    soup.head.insert(0, charset)


Qui si creano e si inseriscono i meta tag nell'html finale.

`soup.new_tag('meta')`: Crea un nuovo tag meta e lo assegna alla variabile desiderata (description, rights, doi etc.).

Viene poi impostato l'attributo `name` e il `content` del tag.

Fai attenzione specialmente alla *bibliographic citation*, qui abbiamo cambiato l'anno per allinearci con il periodo in cui abbiamo generato i nuovi documenti caricati nell'edizione.


In [ ]:
# REMOVE EMPTY TAGS INSIDE HEAD
def clean_empty_head(soup):
    tags = soup.head.find_all('div') #find all div tag in the head and store them in the tags variable
    for tag in tags: #iterate through the tags variable
        if not tag.find('meta'): #if the tag does not contain a meta tag
            tag.decompose() #remove the tag from the document


# REMOVE EMPTY TAGS INSIDE BODY
def clean_empty_body(soup):
    tags = soup.body.find_all() #find all tags in the body and store them in the tags variable
    for tag in tags:
        if len(tag.get_text(strip=True)) == 0: #if the tag is empty
            tag.decompose() #remove the tag from the document
        elif tag.has_attr('class') and tag['class'] == 'no-abstract': #if the tag has the class 'no-abstract'
            tag.unwrap()  #remove the tag from the document


Qui si puliscono il tag `head` e il tag `body` del documento HTML rimuovendo tutti i tag `div` vuoti, (ovvero quelli che non contengono tag `meta`) e nel body quelli con la classe `no-abstract`.

In [ ]:
# FIX TITLE AND REMOVE TAGS INSIDE IT
def clean_title(soup, spans, metadata):
    title = soup.new_tag('title') #create new variable title

    # THEIR DOCUMENTS ARE EXCEPTIONS!!!
    if metadata['curator'] != 'Caterina Iagnemma' and metadata['curator'] != 'Sofia Confalonieri' and metadata['curator'] != 'Maurizio Cau': #if the curator IS NOT Iagnemma, Confalonieri or Cau

        h3 = soup.find('h3')
        h2 = soup.find('h2')
        h1 = soup.find('h1')
        text = False
        if h3:
            h3.name = 'h1' #h3 became h1
            for span in spans: #iterate through the spans list
                if span in h3: #if the span is inside h3
                    span.unwrap() #remove the span from the document
            text = h3.get_text(strip=True) #extract the text of h2 and assign it to the variable "text" removing starting and closing empty space
        elif h2:
            h2.name = 'h1'
            for span in spans:
                if span in h2:
                    span.unwrap()
            text = h2.get_text(strip=True)
        elif h1:
            for span in spans:
                if span in h1:
                    span.unwrap()
            text = h1.get_text(strip=True)
        else: #if there aren't h3, h2 or h1
            p = soup.find('p') #find the p tag
            if p:
                strong = p.find('strong') #find the strong tag inside the p tag
                if strong:
                    strong.unwrap() #remove the strong tag from the document
                p.name = 'h1' #p became h1
                for span in spans: #iterate through the spans list
                    if span in p:
                        span.unwrap() #remove the span from the document
                text = p.get_text(strip=True) #extract the text of p and assign it to the variable "text" removing starting and closing empty space
        if text: #if text is not empty
            title.append(text) #append the text to the title variable

    else: #if the curator IS Iagnemma, Confalonieri or Cau
        h1 = soup.find('h1') #find the h1 tag
        title.append(h1.get_text(strip=True)) #extract the text of h1 and assign it to the variable "title" removing starting and closing empty space
    soup.head.insert(0, title)  #insert the title in the head of the document

In questa parte si va a prelevare e pulire il testo del titolo del documento.
La funzione Crea un nuovo tag `title` da aggiungere all'head del documento HTML.
Poi controlla il valore del curatore nel dizionario metadata.

Se il curatore non è uno tra Caterina Iagnemma, Sofia Confalonieri o Maurizio Cau:

Cerca i tag `h3`, `h2`, e `h1` nel documento HTML.
Se ne trova uno, lo trasforma in `h1` e rimuove eventuali tag `span` contenuti al suo interno, quindi estrae il testo dal tag e lo salva nella variabile `text`.

Se non trova nessuno dei tag `h3`, `h2`, o `h1`, cerca un tag `p`:

All'interno del tag `p`, rimuove eventuali tag `strong`.
Trasforma il `p` in `h1`, rimuove eventuali tag `span` contenuti al suo interno e estrae il testo.
Se riesce a estrarre del testo, lo aggiunge al tag `title`.

Se il curatore è uno tra Caterina Iagnemma, Sofia Confalonieri o Maurizio Cau, prende semplicemente il testo dal tag `h1` e lo aggiunge al tag `title`.

Infine, inserisce il tag `title` creato all'inizio del documento HTML, all'interno del tag `head`.

In [ ]:
# REMOVE TRASH AND SCRAPS TAGS
def clean_trash_and_scraps(soup):
    try:
        trash_entities = soup.find_all(attrs={'typeof': 'moro:Trash'}) #find the attribute typeof with moro:Trash or moro:Scraps
        scraps_entities = soup.find_all(attrs={'typeof': 'moro:Scraps'})
        total = trash_entities + scraps_entities #add the two lists together
        if len(total) > 0: #if the list is not empty
            for el in total: #iterate through the list
                mentions = soup.find_all(attrs={'resource': el['about']})
                metas = soup.find_all('meta', attrs={'about': el['about']})
                for meta in metas:
                    meta.decompose()
                for mention in mentions:
                    mention.unwrap()
    except: None


La funzione `clean_trash_and_scraps` rimuove tag non desiderati dal documento HTML.


In [ ]:
# FIX CURATOR NOTES AND REMOVE TAGS INSIDE THEM
def clean_curator_notes(soup, spans, dataset_ns, metadata):
    notes_section = soup.find('ol', id='curatorNotes') #find the curator notes
    if notes_section:
        notes = notes_section.find_all(attrs={'typeof': 'moro:Footnote'})
        if notes: #if the list of notes is not empty
            for note in notes:
                note['about'] = dataset_ns + f'{metadata["ident"].replace("_", "")}/v1/' + note['about'][1:] #add the dataset namespace to the about attribute of the note
                note['typeof'] = 'fabio:Comment' #set the typeof attribute of the note to fabio:Comment
                h3 = note.find('h3') #find the h3 tag inside the note and remove them
                if h3:
                    h3.unwrap()
                ps = note.find_all('p') #find all p tags inside the note and remove them
                if ps:
                    for p in ps:
                        for span in spans:
                            if span in p.find_all('span') and 'bibref' not in span['class']: #if the span is inside p and has no class bibref
                                span.unwrap() #remove the span from the document
                        p.unwrap() #remove the p tag from the document


La funzione `clean_curator_notes` pulisce e modifica le note del curatore nel documento HTML, rimuovendo tag non necessari e aggiornando gli attributi.

In [ ]:
# FIX MORO NOTES AND REMOVE TAGS INSIDE THEM
def clean_moro_notes(soup, spans, dataset_ns, metadata):
    notes_section = soup.find('ol', id='moroNotes')
    if notes_section:
        notes = notes_section.find_all('li')
        if notes:
            for note in notes:
                note['about'] = dataset_ns + f'{metadata["ident"].replace("_", "")}/v1/' + note['about'][1:]
                note['typeof'] = 'fabio:Comment'
                del note['data-toggle']
                del note['data-placement']
                del note['title']
                del note['data-original-title']
                del note['class']
                ps = note.find_all('p')
                if ps:
                    for p in ps:
                        for span in spans:
                            if span in p.find_all('span') and 'bibref' not in span['class']:
                                span.unwrap()
                        p.unwrap()

                for child in note.children:
                    if child.string:
                        fixed_text = child.string.replace('[', '').replace(']', '')
                        child.string.replace_with(fixed_text)

        del notes_section['data-alert']